# Simple Deep Learning Example Using tf v.1
This notebook denotes a simple code to implement **TensorFlow v.1** to learn and to predict number based on MNIST. 
1. Run the cell below to import some libraries and functions to plot them
    - Becasue tensorflow has migrated most of feature to tensorflow v2.0 (which is very close to keras), **tensorflow.compat.v1** is imported to use old tensorflow.

In [1]:
#used tf.compat.v1 to solve that placeholder error in tf v2.0
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np;
import matplotlib.pyplot as plt;

Instructions for updating:
non-resource variables are not supported in the long term


2. Import MNIST database & change label style to one-hot-encoded

In [2]:
mnist = tf.keras.datasets.mnist
(train_set, train_label), (test_set, test_label) = mnist.load_data()
train_set, test_set = train_set/255, test_set/255
(m, h, w) = train_set.shape

y_train = np.zeros((train_label.shape[0], train_label.max()+1), dtype=np.float32)
y_train[np.arange(train_label.shape[0]), train_label] = 1
y_test = np.zeros((test_label.shape[0], test_label.max()+1), dtype=np.float32)
y_test[np.arange(test_label.shape[0]), test_label] = 1

3. Create the function to call **computational graph** for forward propagation

In [3]:
def conv_forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']

    Z1 = tf.nn.conv2d(input=X, filter=W1, strides=[1,1,1,1], padding = 'SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(value=A1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    Z2 = tf.nn.conv2d(input=P1, filter=W2, strides=[1,1,1,1], padding = 'SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(value=A2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    F = tf.layers.flatten(inputs=P2, data_format='channels_last')
    Z3 = tf.layers.dense(inputs=F, units=10)

    return Z3

4. Run the model: **details are as comments**

In [4]:
# Reset the computational graph
tf.reset_default_graph()

# Reshape the train/test set
train_set_tmp = np.expand_dims(train_set, axis=-1)
test_set_tmp = np.expand_dims(test_set, axis=-1)
train_label_tmp = y_train

# Set leaning parameters
epoch=20
lr = 0.01

# Define model's variables
X = tf.placeholder(dtype="float", shape=(None, h, w, 1))
Y = tf.placeholder(dtype="float", shape=(None, 10))
W1 = tf.get_variable(shape=(2,2,1,3), dtype="float", name="W1", initializer=tf.initializers.glorot_uniform())
W2 = tf.get_variable(shape=(2,2,3,8), dtype="float", name="W2", initializer=tf.initializers.glorot_uniform())
parameters = {"W1": W1, "W2": W2}

# Define the computational objects
Z3 = conv_forward_propagation(X, parameters)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss=cost)
init = tf.global_variables_initializer()

# Start learning session
with tf.Session() as sess:
    sess.run(init)
    for e in range(epoch):
        _ , temp_cost = sess.run(fetches=[optimizer, cost], feed_dict={X: train_set_tmp, Y: train_label_tmp})
        pred_y = sess.run(Z3, feed_dict={X: test_set_tmp})
        accuracy = np.sum(np.argmax(pred_y, axis=1) == test_label)/len(pred_y)
        print("cost= {:1.6f}, accuracy= {:1.6f} ".format(temp_cost, accuracy))

Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

cost= 2.299962, accuracy= 0.216900 
cost= 2.273604, accuracy= 0.339800 
cost= 2.244426, accuracy= 0.435300 
cost= 2.210453, accuracy= 0.499300 
cost= 2.170857, accuracy= 0.549200 
cost= 2.125133, accuracy= 0.588500 
cost= 2.072979, accuracy= 0.614300 
cost= 2.014222, accuracy= 0.633400 
cost= 1.948877, accuracy= 0.644000 
cost= 1.877054, accuracy= 0.657400 
cost= 1.798960, accuracy= 0.669200 
cost= 1.714857, accuracy= 0.680300 
cost= 1.625177, accuracy= 0.695700 
cost= 1.530721, accuracy= 0.710700 
cost= 1.433243, accuracy= 0.726300 
cost= 1.334433, accuracy= 0.746700 
cost= 1.235988, accuracy= 0.762800 
cost= 